In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.sql.functions import monotonically_increasing_id

Algoritmo usando SparkMlib

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel

In [3]:
# Palabras clave para tweets positivos y negativos
palabras_clave_positivas = ["privilegio", "maravilloso", "increíble", "genial", "emocionado","calidad"
                           ,"diversidad","paradisíacas","crecimiento","amigable","ideal","acogedora","innovador","emprendimiento",
                           "evolución","innovadores","emprendedor","excelencia","enriquecedora","oasis","enriquecedora","placentera",
                           "brillante","belleza","tesoro","inspiran","joya","fascinado"]




palabras_clave_negativas = ["triste", "decepcionado", "odio", "mal", "falta","desfasados","incompetencia","carece",
                            "corrupción","indignado","pesadilla","deficiente","discriminación","competitivo","muere",
                            "desigualdad","tortura","desalentadora","monotonía"]

In [4]:
# Función para asignar valor 1 si el mensaje contiene palabras clave positivas, 0 si contiene palabras clave negativas, y -1 si no contiene ninguna palabra clave
def clasificar_mensaje(mensaje):
    for palabra in palabras_clave_positivas:
        if palabra in mensaje:
            return 1
    for palabra in palabras_clave_negativas:
        if palabra in mensaje:
            return 0
    return -1

In [5]:
# Crear una sesión de Spark
spark = SparkSession.builder.appName("ClasificadorMensajes").getOrCreate()


ruta_archivo = "file:///home/student/SIC_Big_Data_CDC/Datos/tweetsMejorado.csv"
df = spark.read.csv(ruta_archivo, header=True, inferSchema=True)


In [6]:
udf_clasificar_mensaje = udf(clasificar_mensaje, IntegerType())


In [7]:
# Agregar la nueva columna de clasificación al DataFrame
df = df.withColumn("clasificacion", udf_clasificar_mensaje(df["mensaje"]))

# Mostrar el DataFrame resultante
df.show(5)

+-----------------+--------------------+-----+-------------+
|             user|             mensaje|likes|clasificacion|
+-----------------+--------------------+-----+-------------+
|       vazquezsue|La Universidad de...|  413|            1|
|    samanthasmith|En la Universidad...|  929|            0|
|alexanderreynolds|La Universidad de...|  932|            1|
|        connielee|En la Universitat...|  395|            0|
|   zacharyjackson|La Universidad de...|  119|            0|
+-----------------+--------------------+-----+-------------+
only showing top 5 rows



In [8]:
# Mostrar las primeras 5 filas del DataFrame, mostrando solo las columnas 'usuario', 'mensaje' y 'clasificacion'


In [9]:
df_pandas = df.toPandas()

# Ahora df_pandas es un DataFrame de pandas
print(df_pandas.head())

                user                                            mensaje  \
0         vazquezsue  La Universidad de Murcia se enorgullece de su ...   
1      samanthasmith  En la Universidad de Barcelona no solo encuent...   
2  alexanderreynolds  La Universidad de Santiago de Compostela es un...   
3          connielee  En la Universitat de Lleida no solo encuentro ...   
4     zacharyjackson  La Universidad de Jaén es una institución que ...   

   likes  clasificacion  
0    413              1  
1    929              0  
2    932              1  
3    395              0  
4    119              0  


In [10]:
primera_fila = df_pandas.iloc[555]['mensaje']
# Mostrar la primera fila
print(primera_fila)

La Universitat Pompeu Fabra es un tesoro académico. Sus campus llenos de historia, su compromiso con la sostenibilidad y el ambiente acogedor hacen que estudiar aquí sea una experiencia inolvidable. #UPF #UniversidadSostenible


In [11]:
primera_fila = df_pandas.iloc[555]['clasificacion']
# Mostrar la primera fila
print(primera_fila)

1


In [12]:
filas_con_clasificacion_negativa = df_pandas[df_pandas['clasificacion'] == -1]

# Mostrar las filas con clasificacion igual a -1
print(filas_con_clasificacion_negativa)#-->NO HAY SIN CLASIFICAR

Empty DataFrame
Columns: [user, mensaje, likes, clasificacion]
Index: []


In [13]:
# Crear una sesión de Spark
spark = SparkSession.builder.appName("Conversion").getOrCreate()
# Crear un DataFrame de PySpark a partir del DataFrame de pandas
df_spark = spark.createDataFrame(df_pandas)
# Ahora df_spark es un DataFrame de PySpark
df_spark.show()

+-----------------+--------------------+-----+-------------+
|             user|             mensaje|likes|clasificacion|
+-----------------+--------------------+-----+-------------+
|       vazquezsue|La Universidad de...|  413|            1|
|    samanthasmith|En la Universidad...|  929|            0|
|alexanderreynolds|La Universidad de...|  932|            1|
|        connielee|En la Universitat...|  395|            0|
|   zacharyjackson|La Universidad de...|  119|            0|
|         reedtony|En la Universidad...|  579|            0|
|      vcunningham|Estudiar en la Un...|  681|            1|
|            jyang|En la Universitat...|  988|            0|
|    warnerrichard|En la Universidad...|  861|            1|
|             jlee|La Universidad de...|   15|            0|
|         jeremy09|¡La Universidad d...|  209|            1|
|          paula26|La Universitat de...|  493|            1|
|         brettkim|En la Universidad...|  247|            1|
|       irasmussen|La Un

# Modelo

In [14]:
spark = SparkSession.builder.appName("MLPipeline").getOrCreate()
train_data, test_data = df_spark.randomSplit([0.8, 0.2], seed=42)

In [15]:
print("Contenido del conjunto de prueba (test_data):")
etiquetas_reales_df = test_data.select("mensaje", "clasificacion")
test_data = test_data.drop("clasificacion")
test_data.show()

Contenido del conjunto de prueba (test_data):
+-----------------+--------------------+-----+
|             user|             mensaje|likes|
+-----------------+--------------------+-----+
|alexanderreynolds|La Universidad de...|  932|
|         andrea37|La Universidad de...|  498|
|         andrew17|¡La Universidad d...|  240|
|            bluna|Estudiar en la Un...|  621|
|          brian53|La Universidad Au...|  171|
|    burgesssandra|En la Universidad...|  159|
|       cristina87|Estudiar en la Un...|  949|
|     debbienguyen|En la Universidad...|  758|
|     edwardslaura|En la Universitat...|  558|
|    emilywilliams|¡La Universidad d...|  896|
|    erikabartlett|La Universitat de...|  570|
|           eyoung|¡La Universidad d...|   15|
|        foleytara|La Universidad de...|  183|
|         ginacarr|La Universidad de...|  919|
|            hwood|La Universidad de...|   29|
|         jeanne36|Estudiar en la Un...|   45|
|     josegonzalez|La Universidad de...|  554|
|  matthewslin

In [16]:
# Stage 1: Tokenizer
tokenizer = Tokenizer(inputCol="mensaje", outputCol="palabras")

# Stage 2: HashingTF
hashingTF = HashingTF(inputCol="palabras", outputCol="features")

# Stage 3: Logistic Regression
logistic_regression = LogisticRegression(maxIter=10, regParam=0.01, labelCol="clasificacion")

# Crear el pipeline con las etapas
pipeline = Pipeline(stages=[tokenizer, hashingTF, logistic_regression])


In [17]:
# Define the parameter grid for hyperparameter tuning (if needed)
paramGrid = (ParamGridBuilder()
             .addGrid(logistic_regression.regParam, [0.01, 0.1, 1.0])
             .build())

# Define the binary classification evaluator
evaluator = BinaryClassificationEvaluator(labelCol="clasificacion")

In [18]:
# Create the cross-validator for hyperparameter tuning (optional)
cross_validator = CrossValidator(estimator=pipeline,
                                 estimatorParamMaps=paramGrid,
                                 evaluator=evaluator,
                                 numFolds=3)

In [19]:
# Fit the cross-validator to the training data (or fit the pipeline if not using hyperparameter tuning)
model = cross_validator.fit(train_data)

# Make predictions on the test data
predictions = model.transform(test_data)

2023-07-27 17:32:54,340 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2023-07-27 17:32:54,341 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [29]:
predictions.show()

+-----------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|             user|             mensaje|likes|            palabras|            features|       rawPrediction|         probability|prediction|
+-----------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|alexanderreynolds|La Universidad de...|  932|[la, universidad,...|(262144,[12967,14...|[-5.9030537661207...|[0.00272365340741...|       1.0|
|         andrea37|La Universidad de...|  498|[la, universidad,...|(262144,[4631,148...|[5.38893790162706...|[0.99545393986948...|       0.0|
|         andrew17|¡La Universidad d...|  240|[¡la, universidad...|(262144,[4631,234...|[5.97759847628605...|[0.99747150287746...|       0.0|
|            bluna|Estudiar en la Un...|  621|[estudiar, en, la...|(262144,[7188,234...|[-4.9756731857293...|[0.00685653317589...|       1.0|
|     

In [21]:
predictions_df = predictions


In [22]:
merged_df = predictions_df.join(etiquetas_reales_df, "mensaje")

# Visualizar el DataFrame unido
print("DataFrame unido:")
merged_df

DataFrame unido:


DataFrame[mensaje: string, user: string, likes: bigint, palabras: array<string>, features: vector, rawPrediction: vector, probability: vector, prediction: double, clasificacion: bigint]

##Medir rendimiento

In [23]:
evaluator_binary = BinaryClassificationEvaluator(labelCol="clasificacion")

# Calcular la precisión del modelo
accuracy = evaluator_binary.evaluate(merged_df, {evaluator_binary.metricName: "areaUnderROC"})
print("Precisión del modelo (Area Under ROC):", accuracy)

Precisión del modelo (Area Under ROC): 1.0


In [24]:
evaluator_multiclass = MulticlassClassificationEvaluator(labelCol="clasificacion")

# Calcular el f1-score del modelo
f1_score = evaluator_multiclass.evaluate(merged_df, {evaluator_multiclass.metricName: "f1"})
print("F1-Score del modelo:", f1_score)


F1-Score del modelo: 1.0


In [25]:
#model.save("ModeloEntrenado")
model.write().save("../Modelos/Sentimientos")

In [26]:
modelo_cargado = CrossValidatorModel.load("../Modelos/Sentimientos")

In [27]:
predictions = modelo_cargado.transform(test_data)

In [30]:
predictions.show()

+-----------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|             user|             mensaje|likes|            palabras|            features|       rawPrediction|         probability|prediction|
+-----------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|alexanderreynolds|La Universidad de...|  932|[la, universidad,...|(262144,[12967,14...|[-5.9030537661207...|[0.00272365340741...|       1.0|
|         andrea37|La Universidad de...|  498|[la, universidad,...|(262144,[4631,148...|[5.38893790162706...|[0.99545393986948...|       0.0|
|         andrew17|¡La Universidad d...|  240|[¡la, universidad...|(262144,[4631,234...|[5.97759847628605...|[0.99747150287746...|       0.0|
|            bluna|Estudiar en la Un...|  621|[estudiar, en, la...|(262144,[7188,234...|[-4.9756731857293...|[0.00685653317589...|       1.0|
|     